In [ ]:
# First cell - Installation
!pip install flask pymongo flask-socketio dnspython pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
!python -m pip install "pymongo[srv]"

# Second cell - Complete Application Code
from flask import Flask, render_template_string, request, jsonify
from flask_socketio import SocketIO, emit
from pymongo import MongoClient
from datetime import datetime, timedelta
from pyngrok import ngrok
import urllib.parse

# MongoDB Connection Parameters
MONGO_USERNAME = "rushilcat23"
MONGO_PASSWORD = urllib.parse.quote("1OtDs5v3xhdLXzkz")
MONGO_CLUSTER = "your_cluster_name.mongodb.net"
MONGO_URI = f"mongodb+srv://rushilcat23:<db_password>@medicines.t2vr8.mongodb.net/?retryWrites=true&w=majority&appName=Medicines"

# Initialize MongoDB Client
client = MongoClient(MONGO_URI)
db = client['hospital_inventory']
medicines_collection = db['medicines']

# Flask App Setup
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Hospital Medicine Inventory</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f4f6f9; }
        .status-low { background-color: #fff3cd; color: #856404; }
        .status-sufficient { background-color: #d4edda; color: #155724; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center mb-4">🏥 Hospital Medicine Inventory</h1>
        
        <div class="card mb-4">
            <div class="card-header">Add New Medicine</div>
            <div class="card-body">
                <form id="addMedicineForm">
                    <div class="row">
                        <div class="col-md-3">
                            <input type="text" class="form-control" id="name" placeholder="Medicine Name" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="quantity" placeholder="Quantity" required>
                        </div>
                        <div class="col-md-3">
                            <input type="date" class="form-control" id="expiry" placeholder="Expiry Date" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="threshold" placeholder="Low Stock Alert" required>
                        </div>
                        <div class="col-md-2">
                            <button type="submit" class="btn btn-primary">Add Medicine</button>
                        </div>
                    </div>
                </form>
            </div>
        </div>

        <table class="table table-striped" id="inventoryTable">
            <thead>
                <tr>
                    <th>Medicine Name</th>
                    <th>Current Stock</th>
                    <th>Expiry Date</th>
                    <th>Low Stock Threshold</th>
                    <th>Status</th>
                    <th>Actions</th>
                </tr>
            </thead>
            <tbody id="inventoryBody"></tbody>
        </table>
    </div>

    <script>
        document.getElementById('addMedicineForm').addEventListener('submit', function(e) {
            e.preventDefault();
            const medicineData = {
                name: document.getElementById('name').value,
                quantity: parseInt(document.getElementById('quantity').value),
                expiry: document.getElementById('expiry').value,
                threshold: parseInt(document.getElementById('threshold').value)
            };
            fetch('/add_medicine', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify(medicineData)
            }).then(response => response.json())
              .then(() => location.reload());
        });

        function updateStock(name, change) {
            fetch('/update_stock', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({name, change})
            }).then(response => response.json())
              .then(() => location.reload());
        }

        fetch('/get_inventory')
            .then(response => response.json())
            .then(medicines => {
                const inventoryBody = document.getElementById('inventoryBody');
                medicines.forEach(medicine => {
                    const row = document.createElement('tr');
                    row.classList.add(medicine.status === 'Low Stock' ? 'status-low' : 'status-sufficient');
                    row.innerHTML = `
                        <td>${medicine.name}</td>
                        <td>${medicine.quantity}</td>
                        <td>${medicine.expiry_date}</td>
                        <td>${medicine.low_stock_threshold}</td>
                        <td>${medicine.status}</td>
                        <td>
                            <button onclick="updateStock('${medicine.name}', 1)" class="btn btn-sm btn-success">+</button>
                            <button onclick="updateStock('${medicine.name}', -1)" class="btn btn-sm btn-danger">-</button>
                        </td>
                    `;
                    inventoryBody.appendChild(row);
                });
            });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/add_medicine', methods=['POST'])
def add_medicine():
    data = request.get_json()
    medicine = {
        'name': data['name'],
        'quantity': data['quantity'],
        'expiry_date': data['expiry'],
        'low_stock_threshold': data['threshold'],
        'added_date': datetime.now()
    }
    medicines_collection.insert_one(medicine)
    return jsonify({"status": "success"})

@app.route('/update_stock', methods=['POST'])
def update_stock():
    data = request.get_json()
    medicines_collection.update_one(
        {'name': data['name']},
        {'$inc': {'quantity': data['change']}}
    )
    return jsonify({"status": "success"})

@app.route('/get_inventory')
def get_inventory():
    medicines = list(medicines_collection.find())
    for medicine in medicines:
        medicine['_id'] = str(medicine['_id'])
        medicine['status'] = 'Low Stock' if medicine['quantity'] <= medicine['low_stock_threshold'] else 'Sufficient'
    return jsonify(medicines)

if __name__ == '__main__':
    ngrok.set_auth_token("2rzFBjmbTlrRNic23R1yCckSGYc_6NCGnYEnVDjxDYK8JQLpF")
    public_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel is running at: {public_url}")
    socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

In [ ]:
from flask import Flask, render_template_string, request, jsonify
import sqlite3
from datetime import datetime
from pyngrok import ngrok

# SQLite Database Initialization
DATABASE = 'medicines_inventory.db'

def init_db():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS medicines (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            quantity INTEGER NOT NULL,
            expiry_date TEXT NOT NULL,
            low_stock_threshold INTEGER NOT NULL,
            added_date TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# Initialize DB
init_db()

# Flask App Setup
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Hospital Medicine Inventory</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f4f6f9; }
        .status-low { background-color: #fff3cd; color: #856404; }
        .status-sufficient { background-color: #d4edda; color: #155724; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center mb-4">🏥 Hospital Medicine Inventory</h1>

        <div class="card mb-4">
            <div class="card-header">Add New Medicine</div>
            <div class="card-body">
                <form id="addMedicineForm">
                    <div class="row">
                        <div class="col-md-3">
                            <input type="text" class="form-control" id="name" placeholder="Medicine Name" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="quantity" placeholder="Quantity" required>
                        </div>
                        <div class="col-md-3">
                            <input type="date" class="form-control" id="expiry" placeholder="Expiry Date" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="threshold" placeholder="Low Stock Alert" required>
                        </div>
                        <div class="col-md-2">
                            <button type="submit" class="btn btn-primary">Add Medicine</button>
                        </div>
                    </div>
                </form>
            </div>
        </div>

        <table class="table table-striped" id="inventoryTable">
            <thead>
                <tr>
                    <th>Medicine Name</th>
                    <th>Current Stock</th>
                    <th>Expiry Date</th>
                    <th>Low Stock Threshold</th>
                    <th>Status</th>
                    <th>Actions</th>
                </tr>
            </thead>
            <tbody id="inventoryBody"></tbody>
        </table>
    </div>

    <script>
        document.getElementById('addMedicineForm').addEventListener('submit', function(e) {
            e.preventDefault();
            const medicineData = {
                name: document.getElementById('name').value.trim(),
                quantity: parseInt(document.getElementById('quantity').value),
                expiry: document.getElementById('expiry').value,
                threshold: parseInt(document.getElementById('threshold').value)
            };
            fetch('/add_medicine', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify(medicineData)
            }).then(response => response.json())
              .then(() => {
                  fetchInventory();  // Fetch updated inventory
              });
        });

        function updateStock(name, change) {
            fetch('/update_stock', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({name, change})
            }).then(response => response.json())
              .then(() => {
                  fetchInventory();  // Fetch updated inventory
              });
        }

        function fetchInventory() {
            fetch('/get_inventory')
                .then(response => response.json())
                .then(medicines => {
                    const inventoryBody = document.getElementById('inventoryBody');
                    inventoryBody.innerHTML = '';  // Clear existing table content
                    medicines.forEach(medicine => {
                        const row = document.createElement('tr');
                        row.classList.add(medicine.status === 'Low Stock' ? 'status-low' : 'status-sufficient');
                        row.innerHTML = `
                            <td>${medicine.name}</td>
                            <td>${medicine.quantity}</td>
                            <td>${medicine.expiry_date}</td>
                            <td>${medicine.low_stock_threshold}</td>
                            <td>${medicine.status}</td>
                            <td>
                                <button onclick="updateStock('${medicine.name}', 1)" class="btn btn-sm btn-success">+</button>
                                <button onclick="updateStock('${medicine.name}', -1)" class="btn btn-sm btn-danger">-</button>
                            </td>
                        `;
                        inventoryBody.appendChild(row);
                    });
                });
        }

        // Fetch inventory on page load
        window.onload = fetchInventory;
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/add_medicine', methods=['POST'])
def add_medicine():
    data = request.get_json()
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''
        INSERT INTO medicines (name, quantity, expiry_date, low_stock_threshold, added_date)
        VALUES (?, ?, ?, ?, ?)
    ''', (data['name'], data['quantity'], data['expiry'], data['threshold'], datetime.now()))
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

@app.route('/update_stock', methods=['POST'])
def update_stock():
    data = request.get_json()
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''
        UPDATE medicines
        SET quantity = quantity + ?
        WHERE name = ?
    ''', (data['change'], data['name']))
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

@app.route('/get_inventory')
def get_inventory():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('SELECT * FROM medicines')
    medicines = c.fetchall()
    conn.close()

    result = []
    for medicine in medicines:
        status = 'Low Stock' if medicine[2] <= medicine[4] else 'Sufficient'
        result.append({
            'id': medicine[0],
            'name': medicine[1],
            'quantity': medicine[2],
            'expiry_date': medicine[3],
            'low_stock_threshold': medicine[4],
            'added_date': medicine[5],
            'status': status
        })
    return jsonify(result)

if __name__ == '__main__':
    try:
        ngrok.set_auth_token("2rzFBjmbTlrRNic23R1yCckSGYc_6NCGnYEnVDjxDYK8JQLpF")
        public_url = ngrok.connect(5000).public_url
        print(f" * ngrok tunnel is running at: {public_url}")
        app.run(host='0.0.0.0', port=5000)
    except Exception as e:
        print(f"Error starting ngrok: {e}")


 * ngrok tunnel is running at: https://8817-34-145-79-114.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 10:33:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 10:33:46] "GET /get_inventory HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 10:33:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 10:34:36] "POST /add_medicine HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 10:34:37] "GET /get_inventory HTTP/1.1" 200 -


from flask import Flask, render_template_string, request, jsonify
from pymongo import MongoClient, errors
from datetime import datetime
from pyngrok import ngrok
import certifi
import urllib.parse

# MongoDB Connection Parameters
MONGO_USERNAME = "rushilcat23"
MONGO_PASSWORD = urllib.parse.quote("1OtDs5v3xhdLXzkz")
MONGO_CLUSTER = "medicines.t2vr8.mongodb.net"
MONGO_URI = f"mongodb+srv://rushilcat23:<db_password>@medicines.t2vr8.mongodb.net/?retryWrites=true&w=majority&appName=Medicines"

# Initialize MongoDB Client
try:
    client = MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client['hospital_inventory']
    medicines_collection = db['medicines']
    print("Connected to MongoDB successfully.")
except errors.ConnectionError as e:
    print(f"Error connecting to MongoDB: {e}")
    exit(1)

# Add Sample Medicines to the Database
def add_sample_medicines():
    sample_medicines = [
        {"name": "Paracetamol", "quantity": 50, "expiry_date": "2025-12-31", "low_stock_threshold": 10},
        {"name": "Amoxicillin", "quantity": 30, "expiry_date": "2024-06-15", "low_stock_threshold": 5},
        {"name": "Cough Syrup", "quantity": 20, "expiry_date": "2023-11-20", "low_stock_threshold": 8},
        {"name": "Ibuprofen", "quantity": 15, "expiry_date": "2025-08-10", "low_stock_threshold": 7}
    ]

    try:
        # Insert the sample medicines
        medicines_collection.insert_many(sample_medicines)
        print("Sample medicines added successfully.")
    except Exception as e:
        print(f"Error inserting sample medicines: {e}")

# Add sample medicines
add_sample_medicines()

# Flask App Setup
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Hospital Medicine Inventory</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f4f6f9; }
        .status-low { background-color: #fff3cd; color: #856404; }
        .status-sufficient { background-color: #d4edda; color: #155724; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center mb-4">🏥 Hospital Medicine Inventory</h1>
        
        <div class="card mb-4">
            <div class="card-header">Add New Medicine</div>
            <div class="card-body">
                <form id="addMedicineForm">
                    <div class="row">
                        <div class="col-md-3">
                            <input type="text" class="form-control" id="name" placeholder="Medicine Name" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="quantity" placeholder="Quantity" required>
                        </div>
                        <div class="col-md-3">
                            <input type="date" class="form-control" id="expiry" placeholder="Expiry Date" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="threshold" placeholder="Low Stock Alert" required>
                        </div>
                        <div class="col-md-2">
                            <button type="submit" class="btn btn-primary">Add Medicine</button>
                        </div>
                    </div>
                </form>
            </div>
        </div>

        <table class="table table-striped" id="inventoryTable">
            <thead>
                <tr>
                    <th>Medicine Name</th>
                    <th>Current Stock</th>
                    <th>Expiry Date</th>
                    <th>Low Stock Threshold</th>
                    <th>Status</th>
                    <th>Actions</th>
                </tr>
            </thead>
            <tbody id="inventoryBody"></tbody>
        </table>
    </div>

    <script>
        document.getElementById('addMedicineForm').addEventListener('submit', function(e) {
            e.preventDefault();
            const medicineData = {
                name: document.getElementById('name').value.trim(),
                quantity: parseInt(document.getElementById('quantity').value),
                expiry: document.getElementById('expiry').value,
                threshold: parseInt(document.getElementById('threshold').value)
            };
            fetch('/add_medicine', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify(medicineData)
            }).then(response => response.json())
              .then(() => {
                  fetchInventory();  // Fetch updated inventory
              });
        });

        function updateStock(name, change) {
            fetch('/update_stock', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({name, change})
            }).then(response => response.json())
              .then(() => {
                  fetchInventory();  // Fetch updated inventory
              });
        }

        function fetchInventory() {
            fetch('/get_inventory')
                .then(response => response.json())
                .then(medicines => {
                    const inventoryBody = document.getElementById('inventoryBody');
                    inventoryBody.innerHTML = '';  // Clear existing table content
                    medicines.forEach(medicine => {
                        const row = document.createElement('tr');
                        row.classList.add(medicine.status === 'Low Stock' ? 'status-low' : 'status-sufficient');
                        row.innerHTML = `
                            <td>${medicine.name}</td>
                            <td>${medicine.quantity}</td>
                            <td>${medicine.expiry_date}</td>
                            <td>${medicine.low_stock_threshold}</td>
                            <td>${medicine.status}</td>
                            <td>
                                <button onclick="updateStock('${medicine.name}', 1)" class="btn btn-sm btn-success">+</button>
                                <button onclick="updateStock('${medicine.name}', -1)" class="btn btn-sm btn-danger">-</button>
                            </td>
                        `;
                        inventoryBody.appendChild(row);
                    });
                });
        }

        // Fetch inventory on page load
        window.onload = fetchInventory;
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/add_medicine', methods=['POST'])
def add_medicine():
    data = request.get_json()
    medicine = {
        'name': data['name'],
        'quantity': data['quantity'],
        'expiry_date': data['expiry'],
        'low_stock_threshold': data['threshold'],
        'added_date': datetime.now()
    }
    medicines_collection.insert_one(medicine)
    return jsonify({"status": "success"})

@app.route('/update_stock', methods=['POST'])
def update_stock():
    data = request.get_json()
    medicines_collection.update_one(
        {'name': data['name']},
        {'$inc': {'quantity': data['change']}}
    )
    return jsonify({"status": "success"})

@app.route('/get_inventory')
def get_inventory():
    medicines = list(medicines_collection.find())
    for medicine in medicines:
        medicine['_id'] = str(medicine['_id'])  # Convert ObjectId to string
        medicine['status'] = 'Low Stock' if medicine['quantity'] <= medicine['low_stock_threshold'] else 'Sufficient'
    return jsonify(medicines)

if __name__ == '__main__':
    try:
        ngrok.set_auth_token("2rzFBjmbTlrRNic23R1yCckSGYc_6NCGnYEnVDjxDYK8JQLpF")
        public_url = ngrok.connect(5000).public_url
        print(f" * ngrok tunnel is running at: {public_url}")
        app.run(host='0.0.0.0', port=5000)
    except Exception as e:
        print(f"Error starting ngrok: {e}")
